In [29]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pandas.io.json import json_normalize
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

### Store CSV into DataFrame

In [30]:
# https://www.kaggle.com/jayrav13/unemployment-by-county-us#output.csv
csv_file = "Resources/Unempoyment_rate.csv"
Unemployment_rate_df = pd.read_csv(csv_file)
Unemployment_rate_df=Unemployment_rate_df.loc[Unemployment_rate_df['Year']==2015]
Unemployment_rate_df=Unemployment_rate_df[Unemployment_rate_df.State != 'New York']
Unemployment_rate_df=Unemployment_rate_df.drop(['Year'], axis=1)
Unemployment_rate_df=Unemployment_rate_df.rename(columns={"Rate": "Monthly_rate"})
Unemployment_rate_df.head()

,Month,State,County,Monthly_rate
0,February,Mississippi,Newton County,6.1
1,February,Mississippi,Panola County,9.4
2,February,Mississippi,Monroe County,7.9
3,February,Mississippi,Hinds County,6.1
4,February,Mississippi,Kemper County,10.6


In [31]:
# Unemployment_rate_df.info()

In [32]:
# https://www.kaggle.com/muonneutrino/us-census-demographic-data
### Data for 2015 for Population
csv_file = "Resources/acs2015_county_data.csv"
county_pop_df = pd.read_csv(csv_file)
# county_data_df.head()

In [33]:
###Create new data with select columns
### Data for 2015
county_col=["State","County","TotalPop","Men","Women"]
county_pop_df = county_pop_df[county_col].copy()

# Set index
# county_transformed.set_index("State", inplace=True)
county_pop_df.reset_index()
county_pop_df.head()

,State,County,TotalPop,Men,Women
0,Alabama,Autauga,55221,26745,28476
1,Alabama,Baldwin,195121,95314,99807
2,Alabama,Barbour,26932,14497,12435
3,Alabama,Bibb,22604,12073,10531
4,Alabama,Blount,57710,28512,29198


In [34]:
# county_transformed.info()

In [35]:
# Unemployment_rate_csv=pd.merge(Unemployment_rate_df,county_pop_df, on=('County'))
# Unemployment_rate_csv=Unemployment_rate_csv.drop(['State_y'], axis=1)
# Unemployment_rate_csv=Unemployment_rate_csv.rename(columns={"State_x": "State"})
# Unemployment_rate_csv.head()

## Store JSON data into a DataFrame

In [36]:
json_file = "Resources/Unempoyment_yearlry_rate.json"
# print(json_file)

In [37]:
filepath = json_file
with open(filepath) as jsonfile:
    json_norm = json.load(jsonfile)
# print(json_norm)

In [38]:
# json_normalize(json_norm)
# print(json.dumps(json_norm, indent=4))


In [39]:
json_2015 = json_norm['2015']['Annual']
# print(json.dumps(json_2015, indent=4))



In [40]:
data={}
state_list=[]
for state in json_2015:
    
    data[state]=[]
    for county in json_2015[str(state)]['Unemployment Rate'].items():
        data[state].append(county)
        
for state in json_2015:
    state_list.append(state)
    
print( data[state])

[('Albany County', 3.1), ('Big Horn County', 4.3), ('Campbell County', 3.8), ('Carbon County', 3.6), ('Converse County', 3.6), ('Crook County', 3.4), ('Fremont County', 5.4), ('Goshen County', 3.2), ('Hot Springs County', 4.1), ('Johnson County', 4.8), ('Laramie County', 3.9), ('Lincoln County', 4.6), ('Natrona County', 4.8), ('Niobrara County', 2.9), ('Park County', 4.2), ('Platte County', 3.8), ('Sheridan County', 4.1), ('Sublette County', 5.1), ('Sweetwater County', 4.6), ('Teton County', 3.8), ('Uinta County', 4.9), ('Washakie County', 4.0), ('Weston County', 3.3)]


In [41]:
annual_rate_df=pd.DataFrame(data[state], columns=('County','Annual_rate'))
annual_rate_df.head()

,County,Annual_rate
0,Albany County,3.1
1,Big Horn County,4.3
2,Campbell County,3.8
3,Carbon County,3.6
4,Converse County,3.6


In [42]:
# Unemployment_final_df=pd.merge(Unemployment_rate_merged,annual_rate_df, on=('County'), how='outer')
# Unemployment_rate_merged=pd.merge(Unemployment_rate_df,county_transformed, on=('County'))
# Unemployment_rate_merged=Unemployment_rate_merged.drop(['State_y'], axis=1)
# Unemployment_final_df.head()

In [43]:
# Unemployment_final_df.info()

### Connect to database

In [46]:
connection_string = "postgres:postgres@artinaysa@3321N:5432/ETL_db"
engine = create_engine(f'postgresql://{connection_string}')

In [48]:
# # Confirm tables
# engine.table_names()

### Load DataFrame into database

In [50]:
Unemployment_rate_df.to_sql(name='unemployment_rate', con=engine, if_exists='append', index=True)

In [ ]:
county_pop_df.to_sql(name='county_pop', con=engine, if_exists='append', index=True)

In [ ]:
annual_rate_df.to_sql(name='annual_rate', con=engine, if_exists='append', index=True)

### Confirm data has been added to database

In [ ]:
pd.read_sql_query('select * from unemployment_rate', con=engine).head()

In [ ]:
pd.read_sql_query('select * from county_pop', con=engine).head()

In [ ]:
pd.read_sql_query('select * from annual_rate', con=engine).head()